# Набор данных о кино-индустрии
- Предметная область: Индустрия кино. Анализ финансовых показателей проката, популярности жанров, прогнозирование доходов
- Источник: https://www.kaggle.com/datasets/danielgrijalvas/movies
- Тип данных: Реальные данные 7000+ фильмов за 1986-2020 года с ресурса IMDb

Набор применим для:
- Регрессии: Прогнозирования доходности проката фильма на основе доступных данных
- Кластеризация: по бюджету и дохода фильма, по рейтингу

Признаки:

| Тип данных | Признак | Описание                     |
|------------|---------|------------------------------|
| name       | string  | Название фильма              |
| rating     | string  | Возрастной рейтинг           |
| genre      | string  | Жанр                         |
| year       | int64   | Год выпуска                  |
| released   | string  | Дата и место первого проката |
| score      | float64 | Рейтинг IMDb                 |
| votes      | float64 | Количество голосов           |
| director   | string  | Режиссёр                     |
| writer     | string  | Сценарист                    |
| star       | string  | Главный актёр                |
| country    | string  | Страна                       |
| budget     | float64 | Бюджет                       |
| gross      | float64 | Доход                        |
| company    | string  | Продюсер                     |
| runtime    | float64 | Продолжительность в минутах  |

In [220]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import re
from typing import Optional

In [221]:
df = pd.read_csv('raw/movies/movies.csv')
df.drop('released', axis=1)
df.sample(10, random_state=228)

In [222]:
df.info()

Видно, что многие целочисленные признаки считались как вещественные. Поправим это

In [223]:
# Convert real values to integers if necessary.
float_features = df.select_dtypes(include='float64').columns
float_features = float_features.dropna()
for feature in float_features:
    func = lambda x: pd.isna(x) or (pd.notna(x) and x.is_integer())
    if df[feature].apply(func).all():
        df[feature] = df[feature].astype('Int64')

Год выпуска (year) часто не совпадает с годом первого показа из released, хотя последний является более важным признаком.

Заменим значения year на год из released, если возможно, а released можно будет удалить

In [224]:
regex = r'(\w+) (\d+), (\d+) \((.+)\)'
for value in df['released']:
    if not re.match(regex, str(value)):
        print(value)

df[df['released'].isna()]

Имеем значения вида '...{year} ({country})' и два вхождения с NaN, которые вскоре всё равно будут удалены, ведь по ним слишком мало информации

In [225]:
def released_to_year(val_str: str) -> Optional[int]:
    val_str = val_str.split('(')[0].strip()
    return int(val_str[-4:])


func = lambda x: released_to_year(x['released']) if str(x['released']) != 'nan' else x['year']
df['year'] = df.apply(func, axis=1)
df = df.drop('released', axis=1)

In [226]:
# Show how many NaN values are in the table.
columns = df.columns
total_rows = df.shape[0]
no_misses_rows = []
for col in columns:
    nan_count = df[col].isna().sum()
    not_empty = df[col].count()
    if not_empty == total_rows:
        no_misses_rows.append(col)
    else:
        print(f'{col}: NaN={nan_count}, Values={not_empty} of {total_rows} rows')
print(f'Cols without any NaN/Empty: {', '.join(no_misses_rows)}')

In [227]:
# Show distribution between budget & year.
plt.figure(figsize=(10, 6))
sns.scatterplot(data=df, x='year', y='budget', color='yellow')
sns.lineplot(data=df, x='year', y='budget', color='red')
plt.ylabel('Budget (mil)')
plt.show()

С годами бюджет фильмов растёт. Попробуем заменить NaN на медианные значения бюджета, опираясь на годовые показатели

In [228]:
# Replace budget NaN values 
mean_per_year = df.groupby('year')['budget'].transform('mean').astype('Int64')
df['budget'] = df['budget'].fillna(mean_per_year)
print('NaN values left:', df.isna().sum().sum())
df.isna().sum().sort_values(ascending=False)

Записи с неопределенными показателями дохода можно удалить за ненадобностью, останется относительно мало включений с NaN - их тоже можно удалить

In [229]:
df = df.dropna()
print('NaN values left:', df.isna().sum().sum())

### Анализ и визуализация

In [230]:
df.sample(10)

In [231]:
# G - General Audiences (no restrictions).
# PG - Parental Guidance Suggested
# PG-13 - 13+
# R - 17+
# NC-17 - 18+

data = df['rating'].value_counts()
data = round(data / sum(data) * 100, 2)

plt.figure(figsize=(10, 8))
ax = sns.barplot(
    x=data.index, y=data,
    hue=data, legend=False
)
plt.ylabel('Percent')
plt.title(f'Movies Rating Popularity')
for ind, val in enumerate(data):
    ax.annotate(
        f'{val:.2f}', xy=(ind, val+0.5), ha='center',
        fontsize=12, fontweight='bold'
    )

In [232]:
data = df['genre'].value_counts()
data = round(data / sum(data) * 100, 2)

plt.figure(figsize=(10, 8))
ax = sns.barplot(
    x=data,
    y=data.index,
    hue=data,
    legend=False
)
ax.set_xlim(0, 32)
plt.ylabel('Percent')
plt.title('Movies Genres Popularity')
for i, value in enumerate(data):
    ax.annotate(
        f'{value:.2f}', xy=(value+0.2, i), va='center',
        fontsize=12, fontweight='bold'
    )

In [233]:
data = df['country'].value_counts()
data = round(data / sum(data) * 100, 2)
data = data[:10]

plt.figure(figsize=(10, 8))
ax = sns.barplot(
    x=data,
    y=data.index,
    hue=data,
    legend=False
)
ax.set_xlim(0, 77)
plt.ylabel('Percent')
plt.title('Movies Countries Popularity')
for i, value in enumerate(data):
    ax.annotate(
        f'{value:.2f}', xy=(value+0.2, i), va='center',
        fontsize=12, fontweight='bold'
    )

In [234]:
data = df['year'].value_counts()

plt.figure(figsize=(10, 4))
ax = sns.lineplot(x=data.index, y=data, color='yellow')
plt.title('Movies Release Year distribution')

In [235]:
data = df['score']

plt.figure(figsize=(10, 6))
plt.title('Movies Score distribution')

# Histogram
bins = 10 * (data.max() - data.min()) + 1
sns.histplot(data, kde=True, bins=int(bins))

# Median and Mean lines
mean, med = data.mean(), data.median()
plt.axvline(mean, color='yellow', linestyle='-.', label=f'Mean: {mean:.1f}')
plt.axvline(med, color='red', linestyle='-.', label=f'Med: {med:.1f}')

plt.legend()
plt.show()

print(f'The score data skew: {df["score"].skew(): 0.2f}')

In [236]:
def print_top(df_col: pd.Series, param: str, count: int = 5) -> None:
    print(f'Top {count} {param} by number of movies')
    for k, v in df_col.value_counts().head(count).items():
        print(f'\t{k}: {v}')
    print()

print_top(df['director'], 'directors')
print_top(df['writer'], 'writers')
print_top(df['company'], 'companies')

In [237]:
print('Top 15 movies of all time (by IMDb rating)')
filtered = df['score'].nlargest(15)
df.loc[filtered.index, ['year', 'name', 'score']]

### Зависимости признаков. Матрица корреляции.

In [238]:
df_numeric = df.select_dtypes(include=[np.number])
corr_matrix = df_numeric.corr()

sns.heatmap(corr_matrix, annot=True, cmap='coolwarm', center=0, linewidths=0.5)
plt.title('Correlation Matrix')
plt.show()

Из матрицы видно:
1. Сильную положительную зависимость gross от budget = 0.71
2. Сильную положительную зависимость gross от votes  = 0.63

In [239]:
plt.figure(figsize=(20, 6))

# Plot 1: Gross/Budget.
plt.subplot(1, 2, 1)
sns.scatterplot(x='budget', y='gross', data=df, hue=df['gross'].astype(int), legend=False, palette='viridis')
sns.regplot(x='budget', y='gross', data=df, scatter=False, color='yellow')
plt.title('Gross to budget')

# Plot 2: Gross/Votes
plt.subplot(1, 2, 2)
sns.scatterplot(x='votes', y='gross', data=df, hue=df['gross'].astype(int), legend=False, palette='viridis')
sns.regplot(x='votes', y='gross', data=df, scatter=False, color='yellow')
plt.title('Gross to votes')

plt.show()

Выводы:
1. Чем сильнее растёт бюджет, тем больше доход;
2. Чем больше оценок (тем больше просмотрело людей), тем больше доход.

In [240]:
# New feature: profit.
df['profit'] = df['gross'] - df['budget']
df.sample(10)

### Категоризация, нормализация, подбор числа кластеров K-Means

In [241]:
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.cluster import KMeans


# Encode labels
label_encoder = LabelEncoder()
categorical_features = ['rating', 'genre', 'country']
for feature in categorical_features:
    df[feature] = label_encoder.fit_transform(df[feature])
    
# Get numerical features, normalize
df_numeric = df.select_dtypes(include=[np.number])
scaler = StandardScaler()
df_numeric = scaler.fit_transform(df_numeric)

# Search for optimal k
k_range = range(1, 15)
kmeans = [KMeans(n_clusters=i) for i in k_range]
score = [kmeans[i-1].fit(df_numeric).score(df_numeric) for i in k_range]

### Кластеризация K-Means

In [242]:
# Show kmeans
plt.plot(k_range, score, marker='o', color='yellow')
plt.xlabel('Num of Clusters')
plt.ylabel('Score')
plt.title('Elbow Curve')
plt.show()

In [243]:
optimal_k = 4

kmeans = KMeans(n_clusters=optimal_k)
kmeans.fit(df_numeric)
len(kmeans.labels_)

In [244]:
df['cluster'] = kmeans.labels_
df.head()

In [245]:
plt.figure(figsize=(12,7))
axis = sns.barplot(x=np.arange(0,optimal_k,1),y=df.groupby(['cluster']).count()['budget'].values)
x=axis.set_xlabel("Cluster Number")
y=axis.set_ylabel("Number of movies")


In [246]:
df_numeric = df.select_dtypes(include=[np.number])
df_numeric = df_numeric.groupby('cluster').mean().round(2)

for cluster, count in df['cluster'].value_counts().sort_index().items():
    print(f'Cluster #{cluster} size: {count}')
    
df_numeric

Можно увидеть следующие группы:
- №0 и №2: Два наибольших кластера - большинство хороших фильмов. Первые выделяются хорошими прибылями profit
- №1: Небольшой кластер - фильмы с малыми прибылями
- №3: Наименьший кластер - фильмы с наибольшими финансовыми показателями

Посмотрим на представителей каждой группы.

In [255]:
# Cluster 0
df_view = df.drop(columns=['rating', 'genre', 'country', 'budget', 'gross'])
df_view[df_view['cluster'] == 0].sample(5)

In [254]:
# Cluster 1
df_view[df_view['cluster'] == 1].sample(5)

In [256]:
# Cluster 2
df_view[df_view['cluster'] == 2].sample(5)

In [257]:
# Cluster 3
df_view[df_view['cluster'] == 3].sample(5)

Действительно, можно наблюдать группу самых удачных прокатов, группу самых неудачных и две посередине. Разница между последними двумя в среднем порядке доходов (разница в десяток)